In [37]:
import pandas as pd

In [38]:
def process_data(filepath, cityname='defaultcity'):
    # Step 1: Load the data
    data = pd.read_csv(filepath)

    # Dictionary to hold station name as key and its properties as value
    station_properties = {}

    # Step 2: Iterate through each unique station
    for station_name in data['STATION'].unique():
        station_data = data[data['STATION'] == station_name]

        # Step 3: Check if the station has complete data
        unique_dates = len(station_data['DATE'].unique())
        if unique_dates >= 5000:
            
            # Step 4: Count missing values in "PRCP" column for that station
            missing_prcp = station_data['PRCP'].isnull().sum()
            
            station_properties[station_name] = {
                'missing_count': missing_prcp,
                'unique_dates': unique_dates
            }

    # Step 5: Sort stations based on their missing values and get the top 2 with least missing values
    sorted_stations = sorted(station_properties.items(), key=lambda x: x[1]['missing_count'])[:2]

    # Extracting data of the top 2 stations from the original dataset
    selected_data_1 = data[data['STATION'] == sorted_stations[0][0]]
    selected_data_2 = data[data['STATION'] == sorted_stations[1][0]]

    # Concatenate the two datasets
    combined_data = pd.concat([selected_data_1, selected_data_2], axis=0)

    # Save the concatenated data to a CSV file
    combined_data.to_csv(f"{cityname}_2_selected_Stations.csv", index=False)
    
    # Return the names and properties of the top 2 stations for reference
    result = [(item[0], item[1]['missing_count'], item[1]['unique_dates']) for item in sorted_stations]
    return result

# Sample usage
top_2_stations = process_data('seattle_PRCP.csv', cityname='Seattle')
print(top_2_stations)


[('USW00024233', 1, 5114), ('USC00458508', 2, 5114)]


In [39]:
def fill_and_save_missing_values(filename, cityname='defaultcity'):
    # Step 1: Load the data
    data = pd.read_csv(filename)

    # Step 2: Count the number of missing values in "PRCP" column before filling
    missing_before = data['PRCP'].isnull().sum()

    # Step 3: Fill missing values in "PRCP" column using interpolation method
    data['PRCP'] = data['PRCP'].interpolate(method='linear', limit_direction='both')

    # Step 4: Count the number of missing values in "PRCP" column after filling
    missing_after = data['PRCP'].isnull().sum()

    # Step 5: Save the filled data back to the same CSV file
    data.to_csv(f"{cityname}_2_selected_Stations.csv", index=False)

    # Step 6: Return the counts of missing values before and after filling
    return missing_before, missing_after

# Sample usage
missing_before, missing_after = fill_and_save_missing_values('Seattle_2_selected_Stations.csv', cityname='Seattle')
print(f"Number of missing values before filling: {missing_before}")
print(f"Number of missing values after filling: {missing_after}")


Number of missing values before filling: 3
Number of missing values after filling: 0


In [43]:
data = pd.read_csv('Seattle_2_selected_Stations.csv')
data.shape

(10228, 8)

In [49]:
import pandas as pd

def compute_annual_metrics(filename):
    # Step 1: Load the data
    data = pd.read_csv(filename)

    # Step 2: Convert 'DATE' column to datetime format
    data['DATE'] = pd.to_datetime(data['DATE'])

    # Step 3: Extract year from 'DATE' column
    data['YEAR'] = data['DATE'].dt.year

    # Step 4: Group by 'STATION' and 'YEAR', then compute mean and sum of 'PRDE'
    grouped = data.groupby(['STATION', 'YEAR'])['PRCP'].agg(['mean', 'sum'])

    # Step 5: Display the results in the desired format
    for station_name, sub_data in grouped.groupby(level=0):
        print(f"Station_name: {station_name}")
        overall_mean = 0
        overall_sum = 0
        count_years = 0
        for year, row in sub_data.iterrows():
            print(f"{year[1]}   {row['mean']:.2f}   {row['sum']:.2f}")
            overall_mean += row['mean']
            overall_sum += row['sum']
            count_years += 1
        print(f"Overall   {overall_mean/count_years:.2f}   {overall_sum:.2f}")
        print()

# Sample usage
compute_annual_metrics('Seattle_2_selected_Stations.csv')


Station_name: USC00458508
2003   0.27   97.03
2004   0.30   110.52
2005   0.24   86.64
2006   0.29   106.17
2007   0.28   100.49
2008   0.30   110.65
2009   0.28   100.93
2010   0.31   112.01
2011   0.34   123.44
2012   0.33   121.21
2013   0.29   105.13
2014   0.33   120.26
2015   0.27   97.12
2016   0.29   107.69
Overall   0.29   1499.28

Station_name: USW00024233
2003   0.11   41.78
2004   0.08   31.10
2005   0.10   35.44
2006   0.13   48.42
2007   0.11   38.95
2008   0.08   30.73
2009   0.11   38.44
2010   0.13   46.99
2011   0.10   36.39
2012   0.13   48.26
2013   0.09   32.56
2014   0.13   48.50
2015   0.12   44.83
2016   0.12   45.18
Overall   0.11   567.57

